# Data corrected and non-corrected BDLSTM forecast

In [ ]:
start = time.time()
lstm_model = generator
forecastOrcorrectedForecast = 'forecast'
lforecast = 382
gridx = 0
gridy = 4
initimestep=0
ini_array = X_all[initimestep, :, :]
ini_arrayObs = X_all[initimestep:, :, :]
if lforecast > ini_arrayObs.shape[0]:
    lforecast = ini_arrayObs.shape[0]
forecast = np.empty((0, trun))
correctedForecast = np.empty((0, trun))
for i in np.arange(lforecast):
    atemp = np.squeeze(lstm_model.predict([np.expand_dims(ini_array, axis=0), np.expand_dims(ini_array, axis=0)]))
    a = np.vstack((ini_array, atemp))
    #ini_arrayObs = np.vstack((ini_array, atemp))
    b = ini_arrayObs[i, :, :]

    forecast = np.append(forecast, np.expand_dims(atemp, 0), axis=0)
    if forecastOrcorrectedForecast == 'forecast':
        prediction = forecast
        ini_array = np.vstack((ini_array[1:, :], forecast[-1]))
        #print(ini_array.shape)
    #Optimal Interpolation
    else:
        amean = np.mean(a, axis=1)
        aamean = a - np.transpose([amean])
        bmean = np.mean(b, axis=1)
        bbmean = b - np.transpose([bmean])
        C = np.matmul(aamean, np.transpose(aamean))
        Cmd = np.matmul(bbmean, np.transpose(aamean))
        # Cmd = 10000
        Cinv = np.linalg.inv(C)
        # Cinv = 1/C
        w = np.matmul(Cinv, aamean)
        Dhat = np.transpose([bmean]) + np.matmul(Cmd, w)

        tempDA = Dhat[-1, :]
        correctedForecast = np.append(correctedForecast, np.expand_dims(tempDA, 0), axis=0)
        prediction = correctedForecast
        ini_array = np.vstack((ini_array[1:, :], tempDA))

print(time.time() - start)

In [ ]:
prediction = inverseScalerThetis(prediction, min_ls, max_ls, min, max)
prediction = np.vstack((pcs_stag[look_backX+initimestep, :15], prediction))

In [ ]:
prediction_masked = np.zeros((prediction.shape[0], csv2.shape[1]))
trun_dataL = np.matmul(prediction, eofs_trun)
sStrunL = trun_dataL[:, 20*0:20*1]*sSsigma + sSmean
sEtrunL = trun_dataL[:, 20*1:20*2]*sEsigma + sEmean
sItrunL = trun_dataL[:, 20*2:20*3]*sIsigma + sImean
sRtrunL = trun_dataL[:, 20*3:20*4]*sRsigma + sRmean

mStrunL = trun_dataL[:, 20*4:20*5]*mSsigma + mSmean
mEtrunL = trun_dataL[:, 20*5:20*6]*mEsigma + mEmean
mItrunL = trun_dataL[:, 20*6:20*7]*mIsigma + mImean
mRtrunL = trun_dataL[:, 20*7:20*8]*mRsigma + mRmean

prediction = np.hstack([sStrunL, sEtrunL, sItrunL, sRtrunL, mStrunL, mEtrunL, mItrunL, mRtrunL])
for i in range(prediction.shape[0]):
    test = np.zeros(csv2.shape[1])
    test[np.where(mask == 1)[1]] = prediction[i, :]
    prediction_masked[i, :] = test
prediction_shaped = np.reshape(prediction_masked,(prediction.shape[0], ngroups, ngridx, ngridy))